In [1]:
import torch
from torchvision import models, transforms
from PIL import Image
import numpy as np
import pandas as pd
import os
from tqdm import tqdm

def procesar_imagenes_y_guardar_csv(lista_paths, archivo_salida):
    # Cargar el modelo ResNet50 preentrenado de PyTorch
    modelo_resnet = models.resnet50(pretrained=True)
    modelo_resnet.eval()  # Poner el modelo en modo de evaluación

    # Definir las transformaciones necesarias para las imágenes
    transform = transforms.Compose([
        transforms.Resize(256),  # Redimensionar la imagen
        transforms.CenterCrop(224),  # Recortar al centro para 224x224
        transforms.ToTensor(),  # Convertir la imagen a tensor
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalizar
    ])

    # Lista para almacenar los vectores de características
    vectores = []

    # Ignorar el primer elemento de la lista de rutas
    lista_paths = lista_paths[1:]  # Esto omite el primer elemento

    for path in tqdm(lista_paths):
        path = "../imgs/" + path
        # Verificar que el archivo existe
        if not os.path.exists(path):
            print(f"El archivo {path} no existe.")
            continue

        # Cargar la imagen y aplicar las transformaciones
        img = Image.open(path).convert('RGB')  # Asegurarse de que esté en RGB
        img_tensor = transform(img).unsqueeze(0)  # Convertir a tensor y agregar dimensión de batch

        # Obtener el vector de características de 1000 dimensiones
        with torch.no_grad():  # Desactivar el cálculo de gradientes (no necesario para inferencia)
            output = modelo_resnet(img_tensor)  # Realizar la predicción
            vector = output.flatten().numpy()  # Aplanar el vector y convertirlo a numpy
            vectores.append(vector)

    # Guardar los vectores en un archivo CSV
    df = pd.DataFrame(vectores)
    df.to_csv(archivo_salida, index=False, header=False)
    print(f"Los vectores se han guardado en {archivo_salida}")

# Ejemplo de uso:
test = pd.read_csv("../X_test_paths.csv").iloc[:, 0]  # Leer la primera columna
train = pd.read_csv("../X_train_paths.csv").iloc[:, 0]  # Leer la primera columna
val = pd.read_csv("../X_val_paths.csv").iloc[:, 0]  # Leer la primera columna

out_test = "../X_test_resnet.csv"
out_train = "../X_train_resnet.csv"
out_val = "../X_val_resnet.csv"




In [2]:
procesar_imagenes_y_guardar_csv(test, out_test)


/Users/dan/miniconda3/envs/ML/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/dan/miniconda3/envs/ML/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
100%|██████████| 7999/7999 [05:19<00:00, 25.01it/s]


Los vectores se han guardado en ../X_test_resnet.csv


In [3]:
procesar_imagenes_y_guardar_csv(train, out_train)


100%|██████████| 64799/64799 [50:50<00:00, 21.24it/s]  


Los vectores se han guardado en ../X_train_resnet.csv


In [4]:
procesar_imagenes_y_guardar_csv(val, out_val)


100%|██████████| 7199/7199 [05:27<00:00, 21.99it/s]


Los vectores se han guardado en ../X_val_resnet.csv
